In [2]:
%matplotlib inline

In [4]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
df_raw = pd.read_table('/home/orange/Desktop/airfoil_self_noise.dat', low_memory=False)
df_raw.head()

,Frequency,AngleOfAttack,ChordLength,StreamVelocity,SideThick,SoundPressureLvl
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [7]:
m = RandomForestRegressor(n_jobs=-1)
df, y, nas = proc_df(df_raw, 'SoundPressureLvl')
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

/home/orange/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9876190600396517

In [8]:
print(m)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)


## Woah! Decision Trees are cool!

In [11]:
df.shape

(1503, 5)

In [12]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 300  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

In [8]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [14]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

/home/orange/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 26.7 ms, sys: 4.56 ms, total: 31.3 ms
Wall time: 109 ms
[0.7361163133972394, 4.292078629766699, 0.9879760653792601, 0.6742336096834551]


### Performance on validation dataset is bad, overfitted

In [6]:
df, y, nas = proc_df(df_raw, 'SoundPressureLvl')
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.30)

In [10]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, oob_score=True, min_samples_leaf=3, max_features=0.3)
m.fit(x_train, y_train)
m.score(x_train, y_train)

0.8770228446679045

In [11]:
m.score(x_test, y_test)

0.7904237346248248

In [12]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=3, max_features=0.5)
m.fit(x_train, y_train)
m.score(x_train, y_train)

0.9228085945799632

In [13]:
m.score(x_test, y_test)

0.8629585649674807